## 독일 INTERSCHUTZ 참여기업 크롤링

### 기업명, 세부정보 링크

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

letter = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','numbers','others']

url_search = 'https://www.interschutz.de/en/expo/exhibitor-index/?letter={}'

co_name_list = []
sc_link_list = []

for l in range(len(letter)):
    url = url_search.format(letter[l])
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        #t1 = soup.select_one('body > main > div > div > div > c-form-frame > div > c-exhibitor-filter > c-exhibitor-list')
        result = soup.find_all('o-exhibitor-snippet')



        for i in range(len(result)):
            # 기업명
            t4 = result[i].find('template',{'slot':'company'})
            co_name = t4.text
            co_name_list.append(co_name)

            # 링크
            t5 = result[i]
            pl = str(t5).split()[1][6:-2] #야매로 처리
            sc_link = 'https://www.interschutz.de' + pl
            sc_link_list.append(sc_link)
    else : 
        print(response.status_code)
# 데이터프레임 생성
df = pd.DataFrame({'co_name':co_name_list,
                'sc_link':sc_link_list})

### 세부정보링크 -> 주소, 국가, 웹사이트, 전화번호 크롤링

In [12]:
import numpy as np
co_desc = []
co_loca = []
co_lono = []
co_cont = []
co_webs = []
co_phnb = []

cnt = len(df['sc_link'])/10
for i in range(len(df['sc_link'])):
    #print(i+1,"/",len(df['sc_link'])," Crawling . . .")
    if i > cnt:
        print(round(i*100/len(df['sc_link'])),"% 완료")
        cnt = cnt + len(df['sc_link'])/10
    response = requests.get(df['sc_link'][i])


    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 기업설명
        try: 
            t1 = soup.find(id="detailExhibitorId4")
            t2 = t1.find('p')
            t3 = str(str(t2).split("<br/>")[1:])[2:-6]
            t4 = t3.replace(",","").replace("&amp","").replace("'","").replace(";","")
            co_desc.append(t4)
        except:
            print("ERR_1! - co_desc",df['sc_link'][i])
            co_desc.append(np.nan)

        # 기업정보
        t1 = soup.find(id="tabs-top")
        t2 = t1.find_all('ul')
        t3 = str(t2).split("</li>")

        try:
            co_loca.append(t3[0][44:]) # 기업 주소
        except:
            print("ERR_2! - co_loca",df['sc_link'][i])
            co_loca.append(np.nan) # 기업 주소
        try:
            co_lono.append(t3[1][5:],) # 기업 우편번호
        except:
            print("ERR_2! - co_lono",df['sc_link'][i])
            co_lono.append(np.nan) # 기업 우편번호
        try:
            co_cont.append(t3[2][5:]) # 기업 소재국가
        except:
            print("ERR_2! - co_cont",df['sc_link'][i])
            co_cont.append(np.nan) # 기업 소재국가
        try:
            co_webs.append(t3[3][20:-32]) # 기업 웹사이트
        except:
            print("ERR_2! - co_webs",df['sc_link'][i])
            co_webs.append(np.nan) # 기업 웹사이트
        try:
            co_phnb.append(t3[4][58:]) # 기업 전화번호
        except:
            print("ERR_2! - co_phnb",df['sc_link'][i])
            co_phnb.append(np.nan) # 기업 전화번호
          
    else :
        print(response.status_code)
    
        
print("Crawling Complete!!!")

df['co_desc'] = co_desc
df['co_loca'] = co_loca
df['co_lono'] = co_lono
df['co_cont'] = co_cont
df['co_webs'] = co_webs
df['co_phnb'] = co_phnb

10 % 완료
ERR_1! - co_desc https://www.interschutz.de/exhibitor/czechtrade/N1487852
20 % 완료
ERR_1! - co_desc https://www.interschutz.de/exhibitor/esteri-group/N1431862
30 % 완료
ERR_2! - co_phnb https://www.interschutz.de/exhibitor/firessense-india/N1491737
ERR_1! - co_desc https://www.interschutz.de/exhibitor/french-pavilion/K113335
ERR_1! - co_desc https://www.interschutz.de/exhibitor/french-pavilion/B531567
ERR_1! - co_desc https://www.interschutz.de/exhibitor/french-pavilion/T803489
ERR_1! - co_desc https://www.interschutz.de/exhibitor/french-pavilion/N1418789
ERR_1! - co_desc https://www.interschutz.de/exhibitor/hannover-milano-fairs-shanghai/N1425405
ERR_1! - co_desc https://www.interschutz.de/exhibitor/hannover-milano-fairs-shanghai/N1431853
40 % 완료
ERR_1! - co_desc https://www.interschutz.de/exhibitor/hannover-fairs-international/N1438447
ERR_1! - co_desc https://www.interschutz.de/exhibitor/hannover-milano-fairs-shanghai/N1418925
ERR_1! - co_desc https://www.interschutz.de/exhibit

### 중복제거

In [15]:
df

,co_name,sc_link,co_desc,co_loca,co_lono,co_cont,co_webs,co_phnb
0,Abel & Käufl Mobilfunkhandel,https://www.interschutz.de/exhibitor/abel-kauf...,As a leading distribution for professional com...,Alter Rennweg 179,84034 Landshut,Germany,http://www.abel-kaeufl.de,+49 871 962150
1,Abilus Group,https://www.interschutz.de/exhibitor/abilus-gr...,Combining European technology with Polish imag...,Przemyslowa 24,32-083 Balice,Poland,http://www.dragonwinch.com,+48 33 4747474
2,accellonet,https://www.interschutz.de/exhibitor/accellone...,accellonet is an owner-managed engineering of...,Marlene-Dietrich-Str. 5,89231 Neu-Ulm,Germany,http://www.accellonet.com,+49 731 9792317-0
3,Access Textiles,https://www.interschutz.de/exhibitor/access-te...,,"Andheri- Kurla Road Bldg. No. 7, 1.Fl,Mittal ...",Mumbai 400059,India,http://www.accesstextiles.com,+91 22 28506396
4,Acco Yangin Ekipmanlari,https://www.interschutz.de/exhibitor/acco-yang...,,Ostim OSB Mah. Ahi Evran Cad. 1227. Sok. 6/E,06374 Ostim - Yenimahalle,Turkey,http://www.accofire.com,+90 312 3852262
...,...,...,...,...,...,...,...,...
1346,5G SMART VENUE,https://www.interschutz.de/exhibitor/5g-smart-...,1.5 million square meters of future - Deutsch...,Messegelände 1,30521 Hannover,Germany,https://www.messe.de/,+49 511 89-0
1347,5G SMART VENUE,https://www.interschutz.de/exhibitor/5g-smart-...,,Messegelände,30521 Hannover,Germany,http://www.messe.de,+49 511 89-0
1348,7 Solutions,https://www.interschutz.de/exhibitor/7-solutio...,,Köhlstr. 14,50827 Köln,Germany,https://www.7solutions.de,+49 221 170894-10
1349,911 Signal Technology,https://www.interschutz.de/exhibitor/911-signa...,,Yuhai East Road 136 Hangzhou Bay New District ...,315199 Ningbo,China,http://www.911signal.com,+86 574 82822561


In [20]:
print("중복 제거 전:",df.shape)
dfs = df.drop_duplicates(['co_name'])
print("중복 제거 후:",dfs.shape)

중복 제거 전: (1351, 8)
중복 제거 후: (1292, 8)


### 출력

In [22]:
import os
dfs.to_csv('interschutz_exhibitor_1.csv', encoding='utf-8')